In [1]:
import pandas as pd
import numpy as np  
#from datetime import datetime
import altair as alt

In [3]:
# from google.colab import drive  #needed to access a file on google drive
# drive.mount('/content/gdrive')

csv_data = pd.read_csv("saved_work/hp_tuning_SVC_poly.csv")
csv_data = pd.concat([csv_data, pd.read_csv("saved_work/hp_tuning_SVC_linear.csv")])
csv_data = pd.concat([csv_data, pd.read_csv("saved_work/hp_tuning_SVC_rbf.csv")])
csv_data = pd.concat([csv_data, pd.read_csv("saved_work/hp_tuning_SVC_sigmoid.csv")])

In [4]:
csv_data

,Result,model_param,Score,type,C,gamma,degree,kernel
0,0.920748,0.03125 / 0.03125 / 3 / poly,Accuracy,train,0.03125,0.031250,3.0,poly
1,0.960561,0.0625 / 0.03125 / 3 / poly,Accuracy,train,0.06250,0.031250,3.0,poly
2,0.880748,0.125 / 0.015625 / 3 / poly,Accuracy,train,0.12500,0.015625,3.0,poly
3,0.987664,0.125 / 0.03125 / 3 / poly,Accuracy,train,0.12500,0.031250,3.0,poly
4,0.920748,0.25 / 0.015625 / 3 / poly,Accuracy,train,0.25000,0.015625,3.0,poly
...,...,...,...,...,...,...,...,...
1479,0.361532,32768.0 / 0.5,F1,test,32768.00000,0.500000,NaN,sigmoid
1480,0.361535,32768.0 / 1.0,F1,test,32768.00000,1.000000,NaN,sigmoid
1481,0.353446,32768.0 / 2.0,F1,test,32768.00000,2.000000,NaN,sigmoid
1482,0.337230,32768.0 / 4.0,F1,test,32768.00000,4.000000,NaN,sigmoid


In [5]:
### How many iterations were done for this model?
len(csv_data.model_param.unique())

427

In [6]:
### look at the "messy" data
df = csv_data.copy()
df = df[df.Score=='F1']

chart = alt.Chart(df).mark_line().encode(
    x = 'model_param:O',
    y = 'Result:Q',
    color = 'type:N',
    tooltip = 'Result:Q',

).properties(
    title = 'F1 Score'
)
chart

alt.Chart(...)

In [7]:
### look at the "messy" data
df = csv_data.copy()
df = df[df.Score=='Accuracy']

chart = alt.Chart(df).mark_line().encode(
    x = 'model_param:O',
    y = 'Result:Q',
    color = 'type:N',
    tooltip = 'Result:Q',

).properties(
    title = 'Accuracy'
)
chart

alt.Chart(...)

In [8]:
### peek at the average results per kernel
df = csv_data.copy()
df = df[df.Score=='F1']
df.groupby(['type','kernel',])['Result'].mean().reset_index()

,type,kernel,Result
0,test,linear,0.401034
1,test,poly,0.357896
2,test,rbf,0.478848
3,test,sigmoid,0.421243
4,train,linear,0.836130
5,train,poly,0.837417
6,train,rbf,0.659497
7,train,sigmoid,0.489699


In [9]:
### peek at the average results per kernel
df = csv_data.copy()
df = df[df.Score=='Accuracy']
df.groupby(['type','kernel',])['Result'].mean().reset_index()

,type,kernel,Result
0,test,linear,0.699209
1,test,poly,0.792336
2,test,rbf,0.721805
3,test,sigmoid,0.649509
4,train,linear,0.917182
5,train,poly,0.937430
6,train,rbf,0.810065
7,train,sigmoid,0.684778


# Looks like Polynomial has the worst results, on average
Let's take a further look at kernel "poly", just in case

In [10]:
df = csv_data.copy()
df = df[ (df.kernel=='poly') ]


chart1 = alt.Chart(df[df.Score=='F1']).mark_line().encode(
    x = 'model_param:O',
    y = 'Result:Q',
    color = 'type:N',
    tooltip = 'model_param:O',
).properties(
    title = 'F1 Score'
)

chart2 = alt.Chart(df[df.Score=='Accuracy']).mark_line().encode(
    x = 'model_param:O',
    y = 'Result:Q',
    color = 'type:N',
    tooltip = 'model_param:O',
).properties(
    title = 'Accuracy'
)

chart1 & chart2

alt.VConcatChart(...)

# OK, now let's look at the trends for each major parameter "all else equal", using the median result of each item.

In [11]:
### F1 chart 
df = csv_data.copy()
df = df[df.Score=='F1']
#print(df.columns)
df = df.groupby(['type','C',])['Result'].median().round(2).reset_index()

# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover', clear='mouseout',
                        fields=['C'], empty='none')

# the basic chart
line = alt.Chart(df).mark_line().encode(
    x = 'C:O',
    y = 'Result:Q',
    color = 'type:N',
    #tooltip = 'C:Q',
)
# ).properties(
#     title = 'F1 Score'
# )
# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(df).mark_point().encode(
    x='C:O',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'Result:Q', alt.value(' '))
)
# Draw a rule at the location of the selection
rules = alt.Chart(df).mark_rule(color='gray').encode(
    x='C:O',
).transform_filter(
    nearest
)

chart1 = alt.layer(line, selectors, points, rules, text).properties()

df = csv_data.copy()
df = df[df.Score=='F1']
df = df.groupby(['type','gamma',])['Result'].median().reset_index()


chart2 = alt.Chart(df).mark_line().encode(
    x = 'gamma:O',
    y = 'Result:Q',
    color = 'type:N',
    tooltip = 'gamma:Q',

).properties(
    title = 'F1 Score'
)

df = csv_data.copy()
df = df[df.Score=='F1']
df = df.groupby(['type','kernel',])['Result'].median().reset_index()

chart3 = alt.Chart(df).mark_line().encode(
    x = 'kernel:O',
    y = 'Result:Q',
    color = 'type:N',
    tooltip = 'Result:Q',

).properties(
    title = 'F1 Score'
)

chart1 | chart2 | chart3

alt.HConcatChart(...)

In [ ]:
df = csv_data.copy()
#df = df[(df.kernel!='poly')]
df = df[df.Score=='Accuracy']
#print(df.columns)
df = df.groupby(['type','C',])['Result'].median().reset_index()

chart1 = alt.Chart(df).mark_line().encode(
    x = 'C:O',
    y = 'Result:Q',
    color = 'type:N',
    tooltip = 'C:Q',

).properties(
    title = 'Accuracy'
)

df = csv_data.copy()
df = df[df.Score=='Accuracy']
#df = df[(df.kernel!='poly')]
df = df.groupby(['type','gamma',])['Result'].median().reset_index()


chart2 = alt.Chart(df).mark_line().encode(
    x = 'gamma:O',
    y = 'Result:Q',
    color = 'type:N',
    tooltip = 'gamma:Q',

).properties(
    title = 'Accuracy'
)

df = csv_data.copy()
df = df[df.Score=='Accuracy']
#df = df[(df.kernel!='poly')]
df = df.groupby(['type','kernel',])['Result'].median().reset_index()

chart3 = alt.Chart(df).mark_line().encode(
    x = 'kernel:O',
    y = 'Result:Q',
    color = 'type:N',
    tooltip = 'kernel:Q',

).properties(
    title = 'Accuracy'
)

chart1 | chart2 | chart3

alt.HConcatChart(...)

## Closer look at all  kernel results of a single kernel

In [ ]:
df = csv_data.copy()
df = df[(df.kernel=='linear') ]


chart1 = alt.Chart(df[df.Score=='F1']).mark_line().encode(
    x = 'model_param:O',
    y = 'Result:Q',
    color = 'type:N',
    tooltip = 'model_param:O',
).properties(
    title = 'F1 Score'
)

chart2 = alt.Chart(df[df.Score=='Accuracy']).mark_line().encode(
    x = 'model_param:O',
    y = 'Result:Q',
    color = 'type:N',
    tooltip = 'model_param:O',
).properties(
    title = 'Accuracy'
)

chart1 & chart2

alt.Chart(...)

#Look again at results, excluding poly and linear and sigmoid

In [47]:
# df = csv_data.copy()
# df = df[(df.kernel!='poly') & (df.kernel!='linear') & (df.kernel!='sigmoid')]
# df = df[df.Score=='Accuracy']
# #print(df.columns)
# df = df.groupby(['type','C',])['Result'].median().reset_index()

# chart1 = alt.Chart(df).mark_line().encode(
#     x = 'C:O',
#     y = 'Result:Q',
#     color = 'type:N',
#     tooltip = 'C:Q',

# ).properties(
#     title = 'Accuracy'
# )

# df = csv_data.copy()
# df = df[df.Score=='Accuracy']
# df = df[(df.kernel!='poly') & (df.kernel!='linear') & (df.kernel!='sigmoid')]
# df = df.groupby(['type','gamma',])['Result'].median().reset_index()


# chart2 = alt.Chart(df).mark_line().encode(
#     x = 'gamma:O',
#     y = 'Result:Q',
#     color = 'type:N',
#     tooltip = 'gamma:Q',

# ).properties(
#     title = 'Accuracy'
# )

# df = csv_data.copy()
# df = df[df.Score=='Accuracy']
# df = df[(df.kernel!='poly') & (df.kernel!='linear') & (df.kernel!='sigmoid')]
# df = df.groupby(['type','kernel',])['Result'].median().reset_index()

# chart3 = alt.Chart(df).mark_line().encode(
#     x = 'kernel:O',
#     y = 'Result:Q',
#     color = 'type:N',
#     tooltip = 'kernel:Q',

# ).properties(
#     title = 'Accuracy'
# )

# chart1 | chart2 | chart3

#notes


In [ ]:
df = csv_data.copy()
df = df[(df.kernel=='rbf') & (df.C==4) & (df.gamma==0.00390625)  ]  
df

,Result,model_param,Score,type,C,gamma,degree,kernel
109,0.896506,4.0 / 0.00390625,Accuracy,train,4.0,0.003906,NaN,rbf
281,0.740139,4.0 / 0.00390625,Accuracy,test,4.0,0.003906,NaN,rbf
453,0.801533,4.0 / 0.00390625,F1,train,4.0,0.003906,NaN,rbf
625,0.490016,4.0 / 0.00390625,F1,test,4.0,0.003906,NaN,rbf
